In [1]:
import math
import glob
import sys 
import cv2 as cv
import pdb
import numpy as np
import os
import copy

In [2]:
#Image show function
def show_image(image, resize=0, window_name='image', timeout=0):
    """
    :param timeout. How many seconds to wait untill it close the window.
    """
    if resize:
        image = resize_image(image, 800, 600)
    cv.imshow(window_name, image)
    cv.waitKey(timeout)
    cv.destroyAllWindows()

#Change the resolution of an image
def resize_image(image, up_width, up_height):
    up_points = (up_width, up_height)
    resized_up = cv.resize(image, up_points, interpolation= cv.INTER_LINEAR)
    return resized_up

In [3]:
# Using SIFT to create the same perspective for the images
def get_keypoints_and_features(image) -> tuple:
    
    """
    :param image.
    :return the keypoints: [cv.Keypoint] and the features: np.ndarray for each keypoint.
    """   

    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) 
    
    sift = cv.SIFT_create() 
    keypoints = sift.detect(gray_image, None)
    keypoints, features = sift.compute(gray_image, keypoints) 
        
    return keypoints, features

def match_features(features_source, features_dest) -> [[cv.DMatch]]:

    """
    Match features from the source image with the features from the destination image.
    :return: [[DMatch]] - The rusult of the matching. For each set of features from the source image,
    it returns the first 'K' matchings from the destination images.
    """
     
    feature_matcher = cv.DescriptorMatcher_create("FlannBased")
    matches = feature_matcher.knnMatch(features_source, features_dest, k=2)   
    return matches

def generate_homography(all_matches:  [cv.DMatch], keypoints_source: [cv.KeyPoint], keypoints_dest : [cv.KeyPoint],
                        ratio: float = 0.75, ransac_rep: int = 4.0):
    """
    class DMatch:
        distance - Distance between descriptors. The lower, the better it is.
        imgIdx - Index of the train image
        queryIdx - Index of the descriptor in query descriptors
        trainIdx - Index of the descriptor in train descriptors
    
    class KeyPoint:
        pt - The x, y coordinates of a point.
    
    """
    if not all_matches:
        return None
    
    matches = [] 
    for match in all_matches:  
        if len(match) == 2 and (match[0].distance / match[1].distance) < ratio:
            matches.append(match[0])
     
    points_source = np.float32([keypoints_source[m.queryIdx].pt for m in matches])
    points_dest = np.float32([keypoints_dest[m.trainIdx].pt for m in matches])

    if len(points_source) > 4:
        H, status = cv.findHomography(points_source, points_dest, cv.RANSAC, ransac_rep)
        return H
    else:
        return None
    
def straighten_image(image_source, image_dest):
    """ 
    :param image_source (image from the right part).
    :param image_dest (image from the left part).
    :param show_details
    :return - the stitched image.
    """    
    keypoints_source, features_source = get_keypoints_and_features(image_source)
    keypoints_dest, features_dest = get_keypoints_and_features(image_dest)
    
    all_matches = match_features(features_source, features_dest)
    
    H = generate_homography(all_matches, keypoints_source, keypoints_dest)    
    result = cv.warpPerspective(image_source, H, (image_dest.shape[1], image_dest.shape[0]))
    return result


In [4]:
#Draw patches from the board
class Line:
    """
    Store a line based on the two points.
    """
    def __init__(self, point_1, point_2):
        self.point_1 = point_1
        self.point_2 = point_2
        

class Point:
    def __init__(self, x, y):
        self.x = np.int32(np.round(x))
        self.y = np.int32(np.round(y))
    
    def get_point_as_tuple(self):
        return (self.x, self.y)
    
        
class Patch:
    """
    Store information about each item (where the mark should be found) in the table. 
    """
    def __init__(self, image_patch, x_min, y_min, x_max, y_max, line_idx, column_idx):
        self.image_patch = image_patch
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        self.line_idx = line_idx
        self.column_idx = column_idx
        self.has_x: int = 0 # 0 meaning it does not contain an 'X', 1 meaning it contains an 'X'
    
    def set_x(self, has_x: int):
        assert has_x == 0 or has_x == 1 # convention 
        self.has_x = has_x
        
def draw_lines(image, lines: [Line], timeout: int = 0, color: tuple = (0, 0, 255),
               return_drawing: bool = False, window_name: str = 'window'):
    """
    Plots the lines into the image.
    :param image.
    :param lines.
    :param timeout. How many seconds to wait untill it close the window.
    :param color. The color used to draw the lines
    :param return_drawing. Use it if you want the drawing to be return instead of displayed.
    :return None if return_drawing is False, otherwise returns the drawing.
    """
    drawing = image.copy()
    if drawing.ndim == 2:
        drawing = cv.cvtColor(drawing, cv.COLOR_GRAY2BGR)
    for line in lines: 
        cv.line(drawing, line.point_1.get_point_as_tuple(), line.point_2.get_point_as_tuple(), color, 2, cv.LINE_AA)
        
    if return_drawing:
        return drawing
    else:
        show_image(drawing,1, window_name=window_name, timeout=timeout)

        
def get_patches(lines: [Line], columns: [Line], image, show_patches: bool = False) -> [Patch]:
    
    """
    It cuts out each box from the table defined by the lines and columns.
    :param lines. The lines that difine the table.
    :param columns. The columns that difine the table.
    :param image. The image containing the table.
    :param show_patches. Determine if the patches will be drawn on the image or not.
    :return : A list with all boxes in the table.
    """
    def crop_patch(image_, x_min, y_min, x_max, y_max):
        """
        Crops the bounding box represented by the coordinates.
        """
        return image_[y_min: y_max, x_min: x_max].copy()
    
    def draw_patch(image_, patch: Patch, color: tuple = (255, 0, 255)):
        """
        Draw the bounding box corresponding to the patch on the image.
        """
        cv.rectangle(image_, (patch.x_min, patch.y_min), (patch.x_max, patch.y_max), color=color, thickness=5)
    
    assert image.ndim == 2
    if show_patches: 
        image_color = np.dstack((image, image, image))
  
    lines.sort(key=lambda line: line.point_1.y)
    columns.sort(key=lambda column: column.point_1.x)
    patches = []
    step = 0
    for line_idx in range(len(lines) - 1):
        for col_idx in range(len(columns) - 1):
            current_line = lines[line_idx]
            next_line = lines[line_idx + 1] 
            
            y_min = current_line.point_1.y + step
            y_max = next_line.point_1.y - step
            
            current_col = columns[col_idx]
            next_col = columns[col_idx + 1]
            x_min = current_col.point_1.x + step 
            x_max = next_col.point_1.x - step
            
            patch = Patch(image_patch=crop_patch(image,  x_min, y_min, x_max, y_max),
                          x_min=x_min, y_min=y_min, x_max=x_max, y_max=y_max,
                          line_idx=line_idx, column_idx=col_idx)
            
            if show_patches:
                draw_patch(image_color, patch)
            
            patches.append(patch)
            
    if show_patches:
        show_image(image_color,1, window_name='patches', timeout=0)
    return patches

def mean_ssd(image_1, image_2) -> float:
    """
     This function receives two matrices having the same dimension and returns the mean of squared differences them.
    :param image_1. The first matrix
    :param image_2. The second matrix
    :return float. The mean of squared differences.
    """ 
    image_1 = np.float32(image_1)  
    image_2 = np.float32(image_2)  
    return np.mean((image_1 - image_2) ** 2)

def get_displacement_vector(img_query, img_template, window_i: tuple = (-15, 15), window_j: tuple = (-15, 15)) -> tuple: 
    """
    It returns (pos_i, pos_j) which is the best alignment of img_query to img_template.
    :param img_query. This is the query image that will be alligned to the template.
    :param img_tempalte. this is the template
    :window_i:tuple. The start and end position on the y axis.
    :window_j:tuple. The start and end position on the x axis.
    :return (pos_i, pos_j) which is the best alignment of img_query to img_template
    """
    pos_i = 0; pos_j = 0
    min_error = np.inf
    height_1, width_1 = img_query.shape        
    height_blue, width_blue = img_template.shape
    
    for i in range(window_i[0], window_i[1]):
        for j in range(window_j[0], window_j[1]):   
            if i >= 0:
                ymin_blue = i
                ymax_blue = height_blue
                ymin_1 = 0
                ymax_1 = height_1 - i
            else:
                ymin_blue = 0
                ymax_blue = height_blue + i
                ymin_1 = -i
                ymax_1 = height_1
               
            if j >= 0:
                xmin_blue = j
                xmax_blue = width_blue
                xmin_1 = 0
                xmax_1 = width_1 - j
            else:
                xmin_blue = 0
                xmax_blue = width_blue + j
                xmin_1 = -j
                xmax_1 = width_1
                
            patch_1 = img_query[ymin_1: ymax_1, xmin_1: xmax_1]
            patch_2 = img_template[ymin_blue: ymax_blue, xmin_blue: xmax_blue]        
            assert patch_1.shape == patch_2.shape
                
            value = mean_ssd(patch_1, patch_2) 
            if min_error > value:
                min_error = value
                pos_i = i
                pos_j = j   
    return pos_i, pos_j

def allign_image_based_on_displacement_vector(image_, pos_i, pos_j):
    """
    Receive the image and the displacement vector and reconstruct the image according to the  displacement vector.
    """
    window_size = max(np.abs([pos_i, pos_j]))
    height, width, _ = image_.shape  
    allign_image = 255 * np.ones((height + window_size * 2, width + window_size * 2, 3), np.uint8) 
    allign_image[pos_i + window_size: pos_i + window_size + height,
                 pos_j + window_size: pos_j + window_size + width] = image_.copy()

    return allign_image

def find_patches_template(image, template, show_intermediate_results: bool = False):
    """
    This function finds the patches of the board in the entire image using the following steps: 
    1. transforming the query image and the template to grayscale
    2. getting the edges for both images using Canny edge detector with threshold1=100 and threshold2=150
    3. obtaining the displacement vector between the query image and the template image
    4. allign the query image to the template
    5. obtain the first left/right corner of the first table and the second table
    6. getting the vertical/horizontal lines based on the table dimension and the number of vertical/horizontal lines based
    7. cut out the paches (the boxes) from the two tables based on the horizontal and vertical lines 
    """
    
    image_gray=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    template_gray=cv.cvtColor(template,cv.COLOR_BGR2GRAY)
    query_image_edges = cv.Canny(image_gray,100,150)
    template_image_edges =  cv.Canny(template_gray,100,150)
    
    if show_intermediate_results: 
        show_image(query_image_edges,1, window_name='query_image_edges', timeout=0)
        show_image(template_image_edges,1, window_name='template_image_edges', timeout=0)
        
    pos_i, pos_j = get_displacement_vector(query_image_edges, template_image_edges)
    alligned_image = allign_image_based_on_displacement_vector(image, pos_i, pos_j)
    
    left_corner_1 = Point(x=22, y=15)               #the points of top left corner of the template image 
    right_corner_1 = Point(x=1885, y=1902)          #the points of lower right corner of the template image 

    step_x =(right_corner_1.x-left_corner_1.x)/15
    step_y =(right_corner_1.y-left_corner_1.y)/15
     
    
    def get_vertical_and_horizontal_lines(left_corner: Point, step_x_, step_y_, window_size_): 
        vertical_lines = []
        horizontal_lines = []
        
        for i in range(0,16):  
            x_min=np.uint64(left_corner.x+i*step_x_+window_size_)
            line = Line(Point(x=x_min, y=0), Point(x=x_min, y=5000))
            vertical_lines.append(line)
        for i in range(0,16):  
            y_min=np.uint64(left_corner.y+i*step_y_+window_size_)
            line = Line(Point(x=0, y=y_min), Point(x=5000, y=y_min))
            horizontal_lines.append(line)   
        
        return vertical_lines, horizontal_lines
     
    
    window_size = np.abs([pos_i, pos_j]).max()
    vertical_lines_left, horizontal_lines_left = get_vertical_and_horizontal_lines(left_corner_1, step_x, step_y, 
                                                                                   window_size_=window_size) 
    
    alligned_image_gray = cv.cvtColor(alligned_image,cv.COLOR_BGR2GRAY)
    patches = get_patches(horizontal_lines_left, vertical_lines_left, alligned_image_gray, 
                               show_patches=show_intermediate_results)
    
    return patches

In [5]:
# Alphabet was used to switch the number of the column with the apropiate letter
import string
alphabet = {1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

#Green_values represents the points placed on the boardgame
green_values = [
    [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5],
    [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
    [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
    [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
    [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
    [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
    [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
    [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
    [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
    [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
    [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5]
]

#Marginal_dominoes_values represents the dominoes situated near the margines of dominoes, because they never changed throughout the assignment, I consider to be easy to remember their order.
#The first values is 7 which represents the start value, to not be mistaken with the value 0 which is a valid position
margin_dominoes_values = [
    7, 1, 2, 3, 4, 5, 6, 0, 2, 5, 3, 4, 6, 2, 2, 0, 3, 5, 4, 1, 6, 2, 4, 5,
    5, 0, 6, 3, 4, 2, 0, 1, 5, 1, 3, 4, 4, 4, 5, 0, 6, 3, 5, 4, 1, 3, 2, 0, 0, 1,
    1, 2, 3, 6, 3, 5, 2, 1, 0, 6, 6, 5, 2, 1, 2, 5, 0, 3, 3, 5, 0, 6, 1, 4, 0, 6,
    3, 5, 1, 4, 2, 6, 2, 3, 1, 6, 5, 6, 2, 0, 4, 0, 1, 6, 4, 4, 1, 6, 6, 3, 0
]


In [6]:
#This is the function where the place, number of a domino and the score are computed

def image_solver(list_patches1,list_patches2,player_moves,turn,score_tracking):   
    list_row = []
    list_col = []
    list_col_letter = []
    list_holes = []
    for i in range(0, len(list_patches1)):
        patch_1 = list_patches1[i].image_patch.copy()
        patch_2 = list_patches2[i].image_patch.copy()

        #Calculate the difference between every patch to determine where a new piece was placed
        diff = cv.absdiff(patch_1, patch_2)
        mean = cv.mean(diff)[0]
        percent_diff = (mean / 255) * 100

        #If the difference is bigger than a certain treshold chosen intentionally 25 to detect only the two patches where a new piece was placed
        if percent_diff > 25:
            row = i//15 + 1                     #compute the row number
            list_row.append(row)
            col = i%15 + 1                      #compute the column number
            list_col.append(col)
            col_letter = alphabet[col].upper()  #replace the number of the column with the proper letter
            list_col_letter.append(col_letter)
            
            thresh = cv.threshold(patch_2, 200, 255, cv.THRESH_BINARY)[1]

            # Apply morphological operations to smooth out the binary image
            kernel = np.ones((5,5), np.uint8)
            morph = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)

            # Detect circles using the Hough Circle Transform algorithm
            circles = cv.HoughCircles(morph, cv.HOUGH_GRADIENT, 1, 10, param1=15, param2=13, minRadius=10, maxRadius=45)

            # Draw the detected circles on the original image
            if circles is not None:
                num_holes = len(circles[0])
                circles = np.round(circles[0, :]).astype("int")
                for (x, y, r) in circles:
                    cv.circle(patch_2, (x, y), r, (0, 255, 0), 2)
                    cv.putText(patch_2, str(num_holes), (x-10, y+10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                num_holes = 0

            list_holes.append(num_holes)

    #Checking if there is a double domino
    if(list_holes[0] == list_holes[1]):
        double_points = 2
    else:
        double_points = 1
    score2 = 0
    #calculate the score, taking the points from the green_values matrix 
    score = double_points*(green_values[list_row[0]-1][list_col[0]-1]+green_values[list_row[1]-1][list_col[1]-1])               
    #score2 is the score that is output in *.txt files        
    score2 = score                                                                                                                      
    
    #Checking if the current player should receive 3 points
    if "player1" in player_moves[turn]:
        if list_holes[0] == margin_dominoes_values[score_tracking[1]] or list_holes[1] == margin_dominoes_values[score_tracking[1]]:
            score2 += 3
    else:
        if list_holes[0] == margin_dominoes_values[score_tracking[2]] or list_holes[1] == margin_dominoes_values[score_tracking[2]]:
            score2 +=3

    #Checking if any player should receive 3 points
    if list_holes[0] == margin_dominoes_values[score_tracking[1]] or list_holes[1] == margin_dominoes_values[score_tracking[1]]:
            score_tracking[1] += 3       
    if list_holes[0] == margin_dominoes_values[score_tracking[2]] or list_holes[1] == margin_dominoes_values[score_tracking[2]]:
            score_tracking[2] += 3

    #Adding the score to score_tracking dictionary
    if "player1" in player_moves[turn]:
        score_tracking[1] += score
    else:
        score_tracking[2] += score
    
    return list_row, list_col_letter, list_holes, score2, score_tracking


In [7]:
def solve_problem(batch_images, player_moves,batch):
    images = []
    # The template board which was used to arrange every image
    image_dest = cv.imread('cropped_board2.jpg')
    #Import all the images of a game in a list 
    for image_path in batch_images:
        image = cv.imread(image_path)
        R=straighten_image(image,image_dest)
        images.append(R)
    
    score_tracking = {1: 0,                                     #initialise the score for every player with 0
                      2: 0}
    
    #Compare the first image with the template image to determine where the first piece is placed
    list_patches1 = find_patches_template(image_dest,image_dest,False)
    list_patches2 = find_patches_template(images[0],image_dest,False)
    list_row, list_col_letter, list_holes, score, score_tracking = image_solver(list_patches1, list_patches2, player_moves,0,score_tracking)
    print(list_row[0],list_col_letter[0]," ",list_holes[0])
    print(list_row[1],list_col_letter[1]," ",list_holes[1])
    print(score)
    print("Player 1 has ",score_tracking[0]," points, PLayer 2 has ",score_tracking[1]," points")

    filename = f"results/{batch}_01.txt"
    with open(filename, "w") as f:
        # write the values to the file
        f.write(f"{list_row[0]}{list_col_letter[0]} {list_holes[0]}\n")
        f.write(f"{list_row[1]}{list_col_letter[1]} {list_holes[1]}\n")
        f.write(f"{score}\n")

    #Compute the location, number and score for every move
    for i in range(0,19):
        list_patches1 = find_patches_template(images[i],image_dest,False)
        list_patches2 = find_patches_template(images[i+1],image_dest,False)
        list_row, list_col_letter, list_holes, score, score_tracking = image_solver(list_patches1, list_patches2, player_moves,i+1,score_tracking)
        filename = f"results/{batch}_{i+2:02d}.txt"
        print(list_row[0],list_col_letter[0]," ",list_holes[0])
        print(list_row[1],list_col_letter[1]," ",list_holes[1])
        print(score)
        print("Player 1 has ",score_tracking[0]," points, PLayer 2 has ",score_tracking[1]," points")
        with open(filename, "w") as f:
            # write the values to the file
            f.write(f"{list_row[0]}{list_col_letter[0]} {list_holes[0]}\n")
            f.write(f"{list_row[1]}{list_col_letter[1]} {list_holes[1]}\n")
            f.write(f"{score}\n")
        

In [8]:
#Main block that generate all the txt files.
#The path for the player moves
def solve_all_images(all_images_path, all_moves_path):
    moves_path = glob.glob(os.path.join(all_moves_path, "*_moves.txt"))
    all_moves_contents =[]
    # Loop through the file list and read in each file
    for filename in moves_path:
        with open(filename, 'r') as file:
            # Do something with the file contents
            file_contents = file.readlines()
            all_moves_contents.append(file_contents)
                
    #The local path for the images path from a single game and 
    all_images_path = glob.glob(os.path.join(all_images_path,"*_*.jpg"))
    batch_size = 20
    for batch in range(0, len(all_images_path), batch_size):
        batch_images = all_images_path[batch:batch+batch_size]
        batch_moves = all_moves_contents[batch//batch_size]
        solve_problem(batch_images, batch_moves, batch//batch_size+1)

In [ ]:
images = "test/regular_tasks"
moves = "test/regular_tasks"
solve_all_images(images,moves)